## Final Assignment 

#### Part 1. 
This AccessViz finds the from the datafolder all the matrices that the user has specified by assigning a list of integer
values that corresponds to YKR_ID found from the MetropAccess_YKR_grid. 
     - AccessViz should inform the user about how the execution process is going. 

##### Find datafiles using os.path.join()-function. 
- This should work for all of the files in the Helsinki Travel Times folder, the user just has to pas the the YKR_IDs as a list of integers, with the seperator ';'. 
- I should also manage to tell the user how the executing process is going for the files. 

In [15]:
import geopandas as gpd
import pandas as pd
import os

#read and take a look at the YKR_grid shp file 
grid_fp = 'data/MetropAccess_YKR_grid/MetropAccess_YKR_grid_EurefFIN.shp'

#read the file
grid = gpd.read_file(grid_fp)

# Change the name of YKR_ID to from_id, to enable the join
grid.rename(columns={'YKR_ID': 'from_id'}, inplace=True)

# See the column names and confirm that we now have a column called 'from_id'
grid.head()
#grid.crs

,x,y,from_id,geometry
0,381875.0,6697880.0,5785640,"POLYGON ((382000.0001388059 6697750.000128186,..."
1,382125.0,6697880.0,5785641,"POLYGON ((382250.00013875 6697750.000128181, 3..."
2,382375.0,6697880.0,5785642,"POLYGON ((382500.0001386951 6697750.000128172,..."
3,382625.0,6697880.0,5785643,"POLYGON ((382750.0001386406 6697750.000128165,..."
4,381125.0,6697630.0,5787544,"POLYGON ((381250.000138978 6697500.000128254, ..."


In [59]:
import glob
import numpy as np
import pandas as pd

#find all txt files in data/TravelTimes folder
filepaths = glob.glob('data/TravelTimes/*.txt')
#print(filepaths)

#list of YKR_IDs I want to import: 
YKR_IDs = ['5975373_Forum.txt', '5878087_Dixi.txt']

#Find filepaths for files specified in YKR_IDs 
for YKR_ID in YKR_IDs: 
    path = ('data/TravelTimes/TravelTimes_to_')
    filepaths = os.path.join(path, YKR_ID)
    
    if YKR_ID in YKR_IDs is True:
        print(filepaths)
    else: 
        print('This YKR_ID does not exist in folder')



    

This YKR_ID does not exist in folder
This YKR_ID does not exist in folder


In [ ]:
if filepaths.contain(id_list):
    print (filepaths)
    else:
        print('This YKR_ID does not exist in folder')

#read the filepaths in filepaths-list  into geodataframe
for idx, fp in enumerate(filepaths):
    #print(fp)
    data = pd.read_csv(fp, sep=';')
    print(fp)

#### PART 2: 
Create Shapefiles from the chosen Matrix text tables (e.g. travel_times_to_5797076.txt) by joining the Matrix file with MetropAccess_YKR_grid Shapefile where from_id in Matrix file corresponds to YKR_ID in the Shapefile. The tool saves the result in the output-folder that user has defined. You should name the files in a way that it is possible to identify the ID from the name (e.g. 5797076).

##### Join the two tables into one geodataframe based on YKR_ID and from_id


In [31]:
'''Replace -1 with NaN values'''
#data = data.replace(to_replace={-1: 999})
    
'''merge data (pd) and grid(gpd) together on 'from_id'  '''
data = data.merge(grid, on='from_id')
#print(data.head())
   
'''Create into geodataframe'''
#set CRS
CRS = grid.crs
#make in into a geodataframe
travel_times = gpd.GeoDataFrame(data, crs=CRS, geometry ='geometry')
#print(travel_times.tail())
    
#check the output
travel_times.head()

,from_id,to_id,walk_t,walk_d,car_r_t,car_r_d,car_m_t,car_m_d,pt_r_t,pt_r_tt,...,pt_m_d,x_x,y_x,geometry_x,x_y,y_y,geometry_y,x,y,geometry
0,5785640,5878087,350,24513,45,27882,40,27890,102,134,...,30241,381875.0,6697880.0,"POLYGON ((382000.0001388059 6697750.000128186,...",381875.0,6697880.0,"POLYGON ((382000.0001388059 6697750.000128186,...",381875.0,6697880.0,"POLYGON ((382000.0001388059 6697750.000128186,..."
1,5785641,5878087,321,22438,45,28093,40,28100,109,130,...,30447,382125.0,6697880.0,"POLYGON ((382250.00013875 6697750.000128181, 3...",382125.0,6697880.0,"POLYGON ((382250.00013875 6697750.000128181, 3...",382125.0,6697880.0,"POLYGON ((382250.00013875 6697750.000128181, 3..."
2,5785642,5878087,322,22518,52,22575,46,22583,111,130,...,30564,382375.0,6697880.0,"POLYGON ((382500.0001386951 6697750.000128172,...",382375.0,6697880.0,"POLYGON ((382500.0001386951 6697750.000128172,...",382375.0,6697880.0,"POLYGON ((382500.0001386951 6697750.000128172,..."
3,5785643,5878087,326,22811,48,23991,41,23992,115,140,...,30857,382625.0,6697880.0,"POLYGON ((382750.0001386406 6697750.000128165,...",382625.0,6697880.0,"POLYGON ((382750.0001386406 6697750.000128165,...",382625.0,6697880.0,"POLYGON ((382750.0001386406 6697750.000128165,..."
4,5787544,5878087,343,24024,44,27456,39,27464,99,118,...,27077,381125.0,6697630.0,"POLYGON ((381250.000138978 6697500.000128254, ...",381125.0,6697630.0,"POLYGON ((381250.000138978 6697500.000128254, ...",381125.0,6697630.0,"POLYGON ((381250.000138978 6697500.000128254, ..."


##### Save the geodataframe into shapefile which has a unique number (the YKR_ID)

In [37]:
travel_times.to_file(filename = 'YKR_ID')

ValueError: Invalid field type <class 'shapely.geometry.polygon.Polygon'>